In [4]:
from typedspark import Schema, Column, StructType
from pyspark.sql.types import IntegerType, StringType

class A(Schema):
    alpha: Column[IntegerType]
    bravo: Column[StringType]

class B(Schema):
    a: Column[StructType[A]]
    b: Column[StringType]

alpha
bravo


In [5]:
B


from pyspark.sql.types import IntegerType, StringType

from typedspark import Column, Schema, StructType


class B(Schema):
    a: Column[StructType[__main__.A]]
    b: Column[StringType]


class A(Schema):
    alpha: Column[IntegerType]
    bravo: Column[StringType]